In [1]:
from flask import Flask, render_template, request, jsonify, redirect, url_for

from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium")

app = Flask(__name__)

# Dummy user data (replace this with a proper user authentication mechanism)
users = {'user1': 'password1', 'user2': 'password2'}

@app.route("/")
def index():
    return render_template('login.html')

@app.route("/login", methods=["POST"])
def login():
    username = request.form['username']
    password = request.form['password']

    if username in users and users[username] == password:
        # Redirect to chat.html after successful login
        return redirect(url_for('chat'))
    else:
        return render_template('login.html', error='Invalid username or password')

# Add this route for rendering chat.html
@app.route("/chat")
def chat():
    return render_template('chat.html')

@app.route("/get", methods=["POST"])
def chat_response():
    msg = request.form["msg"]
    return get_Chat_response(msg)

def get_Chat_response(text):
    # Let's chat for 5 lines
    for step in range(5):
        # encode the new user input, add the eos_token and return a tensor in Pytorch
        new_user_input_ids = tokenizer.encode(str(text) + tokenizer.eos_token, return_tensors='pt')

        # append the new user input tokens to the chat history
        bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids

        # generated a response while limiting the total chat history to 1000 tokens,
        chat_history_ids = model.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)

        # pretty print last output tokens from bot
        response = tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)

    return response

if __name__ == '__main__':
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
